<div class="alert alert-info">
    <center><b>Set up Notebook</b></center>
</div>

In [ ]:
%pip install GitPython pandas tqdm pathlib psycopg2 python-dotenv

In [2]:
from git import Repo
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import pytz
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor, as_completed
import subprocess
import psycopg2
from dotenv import load_dotenv
from io import StringIO

In [3]:
load_dotenv()
DB_PASSWORD = os.getenv('DB_PASSWORD')

<div class="alert alert-info">
    <center><b>Add class representaion of the commits that'll be stored in the DB</b></center>
</div>

In [4]:

@dataclass
class Commit:
    full_path: str
    timestamp: datetime
    sha: str
    message: str
    diff: str
    ecossystem: str
    link: str
    
    def __str__(self):
        return f"-{self.sha}\n- {self.message}\n- {self.timestamp}\n- {self.diff}\n- {self.ecossystem}\n- {self.link}"
    
    def __repr__(self):
        return self.__str__()

<div class="alert alert-info">
    <center><b>Add all methods</b></center>
</div>

In [5]:
def db_conn(db: str, password: str, user: str) -> psycopg2.extensions.connection:
    """Connects to the specified database.
    
    Args:
        db (str) - The name of the database to connect to.\n
        password (str) - The password for the user to connect with.\n
        user (str) - The user to connect with.\n
    
    Returns:
        psycopg2.extensions.connection: The connection object.
    """
    return psycopg2.connect(
        database = db,
        user = user,
        host = 'localhost',
        password = password,
        port = '5432'        
    )

In [6]:
def clone(git_url: str, repo_dir: str, sample: str) -> None:
    '''Clone a git repository and checkout all files in the repository
    
    Args:
        git_url (str) - URL of the git repository\n
        repo_dir (str) - Directory to clone the repository to\n
        sample (str) - Name of the sample\n
        
    Returns:
        None
    '''
    repo_path = os.path.join(repo_dir, sample)
    os.makedirs(repo_path, exist_ok=True)

    repo = Repo.clone_from(git_url, repo_path, multi_options=["--no-checkout"])

    try:
        repo.git.reset('--hard', 'HEAD')

        repo.git.checkout('--', '.')
    except Exception as e:
        print(f"Error checking out files for {sample}: {e}")

In [7]:
def download(sample: str) -> None:
    '''Download the repository. If the repository is already downloaded,
    nothing is done.
    
    Args:
        sample (str) - Name of the sample
    
    Returns:
        None
    '''
    gitHubUrl = f"https://github.com/{sample}.git"
    repoDir = "repositories/"
    isdir = os.path.isdir(repoDir+sample)
    if isdir:
        return
    else:
        clone(gitHubUrl, repoDir, sample)

In [8]:
def add_commit(commit: Commit) -> None:
    conn = db_conn('code_samples', 'codesamples', 'codesamples_user')
    conn.autocommit = True
    cursor = conn.cursor()
    
    cursor.execute(f"""INSERT INTO commits (full_path, timestamp, sha, message, diff, ecossystem, link) VALUES (%s, %s, %s, %s, %s, %s, %s)""",
                (commit.full_path, commit.timestamp, commit.sha, commit.message, commit.diff, commit.ecossystem, commit.link))
    conn.commit()
    
    cursor.close()
    conn.close()

In [9]:
def is_commit_in_db(commit: Commit) -> bool:
    """Checks if a commit is already in the database.
    
    Args:
        commit (Commit) - The commit to check.
        
    Returns:
        bool: True if the commit is in the database, False otherwise.
    """
    conn = db_conn('code_samples', 'codesamples', 'codesamples_user')
    cursor = conn.cursor()
    
    cursor.execute(f"""SELECT 1 FROM commits WHERE sha = '{commit.sha}';""")
    exists = cursor.fetchone()
    
    cursor.close()
    conn.close()
    
    return exists

In [10]:
def add_formatted_commit(commit_text: str, repo_path: str) -> None:
    """Parses the diff of a commit and adds the whole commit to the database.

    Args:
        commit_text (str) - The text of the commit to parse.\n
        repo_path (str) - The path to the repository the commit is from.\n
        
    Returns:
        None
    """
    sha, timestamp, message, diff_lines = None, None, None, []

    for line in StringIO(commit_text):
        if "<<DELIM>>" in line:
            if sha:
                ecossystem = repo_path.split('\\')[1]
                link = f"https://github.com/{ecossystem}/{repo_path.split('\\')[2]}/commit/{sha}"
                commit = Commit(repo_path, datetime.fromtimestamp(int(timestamp), tz=pytz.utc),
                                sha, message, "\n".join(diff_lines), ecossystem, link)
                add_commit(commit)
            sha, timestamp, message = line.split("<<DELIM>>")
            diff_lines = []
        else:
            diff_lines.append(line.strip())

    if sha:
        ecossystem = repo_path.split('\\')[1]
        link = f"https://github.com/{ecossystem}/{repo_path.split('\\')[2]}/commit/{sha}"
        commit = Commit(repo_path, datetime.fromtimestamp(int(timestamp), tz=pytz.utc),
                        sha, message, "\n".join(diff_lines), ecossystem, link)
        add_commit(commit)


In [11]:
def add_all_commits(repo_path: str, cutoff_date: datetime) -> None:
    """Fetches the commits from a repository that were made before a certain 
    date and adds them to the database.

    Args:
        repo_path (str) - The path to the repository to fetch commits from.\n
        cutoff_date (datetime) - The date to fetch commits until.\n
        
    Returns:
        None
    """
    if not os.path.exists(os.path.join(repo_path, '.git')):
        print(f"Skipping non-Git directory: {repo_path}")
        return

    try:
        subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=repo_path)
    except subprocess.CalledProcessError:
        print(f"Skipping invalid Git repository: {repo_path}")
        return

    try:
        process = subprocess.Popen(
            ["git", "log", "--pretty=format:%H<<DELIM>>%ct<<DELIM>>%s", "--patch", f"--until={cutoff_date.timestamp()}"],
            cwd=repo_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        stdout, stderr = process.communicate()

        if process.returncode != 0:
            print(f"Error fetching logs for {repo_path}: {stderr.decode('utf-8')}")
            return

        commit_info = stdout.decode('utf-8', errors='replace')
        add_formatted_commit(commit_info, repo_path)
        

    except Exception as e:
        print(f"Unexpected error processing {repo_path}: {e}")

In [12]:
def get_all_repos_raw_data(parent_folder: str) -> None:
    """Processes all repositories in a parent folder and adds all the commits from them.

    Args:
        parent_folder (str) - The path to the folder containing all repositories.

    Returns:
        None
    """
    repo_paths = []
    
    for sub_dir in os.listdir(parent_folder):
        sub_dir_path = os.path.join(parent_folder, sub_dir)
        if os.path.isdir(sub_dir_path):
            for repo_dir in os.listdir(sub_dir_path):
                repo_dir_path = os.path.join(sub_dir_path, repo_dir)
                if os.path.isdir(repo_dir_path) and os.path.exists(os.path.join(repo_dir_path, '.git')):
                    repo_paths.append(repo_dir_path)

    with ThreadPoolExecutor(max_workers=26) as executor:
        future_to_repo = {executor.submit(add_all_commits, repo, datetime(2024, 9, 19, tzinfo=pytz.UTC)): repo for repo in repo_paths}
        for future in tqdm(as_completed(future_to_repo), total=len(future_to_repo), desc="Processing Repositories"):
            try:
                future.result()
            except Exception as e:
                print(f"Error in future: {e}")

<div class="alert alert-info">
    <center><b>Set up the database</b></center>
</div>

In [13]:
conn = db_conn('postgres', DB_PASSWORD, 'postgres')
cursor = conn.cursor()
conn.autocommit = True

cursor.execute(f"""SELECT 1 FROM pg_catalog.pg_database WHERE datname = 'code_samples';""")
db_exists = cursor.fetchone()

if db_exists:
    cursor.execute("DROP DATABASE code_samples;")
    cursor.execute("DROP OWNED BY codesamples_user CASCADE;")

cursor.execute("CREATE DATABASE code_samples;")

cursor.execute(f"""SELECT 1 FROM pg_roles WHERE rolname = 'codesamples_user';""")
user_exists = cursor.fetchone()

if not user_exists:
    cursor.execute("CREATE USER codesamples_user WITH PASSWORD 'codesamples';")

cursor.execute("GRANT ALL PRIVILEGES ON DATABASE code_samples TO codesamples_user;")

cursor.close()
conn.close()

conn = db_conn('code_samples', DB_PASSWORD, 'postgres')
conn.autocommit = True
cursor = conn.cursor()

cursor.execute("ALTER SCHEMA public OWNER TO codesamples_user;")

cursor.execute("GRANT ALL ON SCHEMA public TO codesamples_user;")
cursor.execute("ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT ALL ON TABLES TO codesamples_user;")

cursor.close()
conn.close()

conn = db_conn('code_samples', 'codesamples', 'codesamples_user')
cursor = conn.cursor()

cursor.execute(f"""CREATE TABLE IF NOT EXISTS commits (
     sha TEXT PRIMARY KEY,
     full_path TEXT,
     ecossystem TEXT,
     link TEXT,
     timestamp TIMESTAMP,
     message TEXT,
     diff TEXT
     );""")

conn.commit()

cursor.execute(f"""CREATE TABLE IF NOT EXISTS files (
    name TEXT,
    sha TEXT,
    type TEXT,
    diff TEXT,
    PRIMARY KEY (name, sha),
    FOREIGN KEY (sha) REFERENCES code_samples.public.commits(sha)
     );""")

conn.commit()

cursor.close()
conn.close()

<div class="alert alert-info">
    <center><b>Download all repositories</b></center>
</div>

In [14]:
repos = pd.read_csv('../code_samples.csv', skiprows=1)
repos = repos.dropna(subset=['html_url'])

In [ ]:
for i in tqdm(range(len(repos)), desc=f"Downloading Repositories"):
    repo = repos.iloc[i]
    repo_ecosystem = repo['html_url'].split('/')[-2]
    repo_name = repo['name']
    sample_name = f"{repo_ecosystem}/{repo_name}"
    download(sample_name)

<div class="alert alert-info">
    <center><b>Extract all commits and populate the database with them</b></center>
</div>

In [ ]:
all_raw_data = get_all_repos_raw_data(os.path.join('repositories'))